In [5]:
'''
Based on: https://github.com/fmfn/BayesianOptimization/blob/master/examples/xgboost_example.py
Computes the best parameters for XGB model optimization
'''

# imports
import pandas as pd
import numpy as np
import os
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit

# data columns used for the booster
factors = ['area_name', 'days_on_market', 'property_id', 'bedrooms', 'bathrooms', 'sqft','longitude', 'latitude','zipcode', 'elevation', 'garage'
                          ,'year_built', 'level','dist_to_park','dist_to_golf_course', 'has_pool'
                          ,'date_closed','multifamily', 'hoa_fees', 'lot', 'zestimate_rent', 'zestimate_sale']

In [2]:
def XGBcv(eta, max_depth, gamma, min_child_weight, max_delta_step, subsample, colsample_bytree, alpha):
    folds = 5
    paramt = {
        'verbose_eval': None,
        'silent': 0,
        'objective': 'reg:linear',
        'booster': 'gbtree',
        'eval_metric': 'mae',
        'updater': 'grow_gpu',
        'eta': max(eta, 0),
        'max_depth': int(max_depth),
        'alpha': max(alpha, 0),
        'gamma': max(gamma, 0),
        'subsample': max(min(subsample, 1), 0),
        'colsample_bytree': max(min(colsample_bytree, 1), 0),
        'min_child_weight': int(min_child_weight),
        'max_delta_step': int(max_delta_step)
    }
    
    out = xgb.cv(paramt,
           dtrain,
           num_boost_round=30000,
           folds=tscv.split(dtrain),
           callbacks=[xgb.callback.early_stop(50)])
    
    return -out['test-mae-mean'].values[-1]


In [3]:
# from past experiments
#init_values = {'target': [-193446, -193446, -24459, -192546, -47780, -37024, -24803, -193746.078125, -193746.06770833334, -25118.120442333337, -193446.38541666666, -25894.672526, -28622.063151, -193446.38541666666, -193446.38541666666, -193446.38541666666, -193446.38541666666, -193446.38541666666, -193146.109375, -192847.13020833334, -193746.078125, -193446.38541666666, -193446.38541666666, -193446.38541666666, -193446.38541666666, -193446.38541666666, -193446.38541666666, -193446.38541666666], 'alpha': [9.99437912773472, 10.0, 10.0, 10.0, 0.0, 0.0, 10.0, 4.015546409782273, 9.926950421589801, 8.896402432054556, 9.99437912773472, 0.5860820301821901, 0.2082503541460734, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.99437912773472, 9.99437912773472, 0.80436438137688415, 9.2037783619021472, 4.0155464097822726, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191, 9.9943791277347191], 'colsample_bytree': [0.26467482205195547, 1.0, 1.0, 0.1, 0.1, 0.1, 1.0, 0.7833851019508481, 0.9322096383814842, 0.3185166699727022, 0.26467482205195547, 0.17611852499412145, 0.2632600769188274, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.86097644716887534, 0.13285454916293565, 0.7833851019508481, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547, 0.26467482205195547], 'gamma': [6.417811142881344, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.135534847780158, 0.2500394735324063, 0.5231786735390687, 6.417811142881344, 0.6138584518188217, 9.986369268602083, 6.417811142881344, 6.417811142881344, 6.417811142881344, 6.417811142881344, 6.417811142881344, 0.2275677738994164, 0.088652374383074717, 5.1355348477801579, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436, 6.4178111428813436], 'max_delta_step': [2.8567339088646717, 2.2602105756123785, 0.0, 5.0, 0.0, 0.0, 0.0, 1.4019764818025178, 1.4850239461478287, 0.46898495419691544, 2.8567339088646717, 0.19645230111910383, 0.62666064228038, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 3.165663024094056, 4.9049044579924264, 1.4019764818025178, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717, 2.8567339088646717], 'max_depth': [9.996819662914094, 5.0, 15.0, 5.0, 15.0, 5.0, 15.0, 6.649297681680842, 6.264494397242032, 14.315316864637497, 9.996819662914094, 14.195801982264026, 5.649088797554254, 9.996819662914094, 9.996819662914094, 9.996819662914094, 9.996819662914094, 9.996819662914094, 5.7563941946454333, 14.886726836368352, 6.6492976816808422, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937, 9.9968196629140937], 'min_child_weight': [6.3381577144228505, 19.494684702216194, 20.0, 1.0, 1.0, 20.0, 1.0, 9.297862559862699, 1.1124257567329519, 19.960599578245827, 6.3381577144228505, 19.57058716847681, 19.934027009976592, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 19.950610038312728, 1.3119667615461024, 9.2978625598626987, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505, 6.3381577144228505], 'subsample': [0.7964743895201556, 0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5057064445599841, 0.892644979559955, 0.638250659261731, 0.7964743895201556, 0.9616614654616669, 0.6059873611138029, 0.7964743895201556, 0.7964743895201556, 0.7964743895201556, 0.7964743895201556, 0.7964743895201556, 0.53720760387562039, 0.65232968466409491, 0.50570644455998415, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559, 0.79647438952015559]}

In [ ]:
params = { 'eta': (0, 0.3),
                 'max_depth': (5, 15),
                 'gamma': (0.0, 10.0),
                 'min_child_weight': (1, 20),
                 'max_delta_step': (0, 5),
                 'subsample': (0.5, 1.0),
                 'colsample_bytree' :(0.1, 1.0),
                 'alpha': (0, 10)
               }

XGB_BOpt = BayesianOptimization(XGBcv, params)
#XGB_BOpt.initialize(init_values)

df = pd.read_csv('CSV_backups/ALL-sales.csv')

# uncomment this to run on a subset of the dataset (for debugging)
#msk = np.random.rand(len(df)) < 0.1  # pick x% of the dataset for a quick run, 100% would be entire dataset
#df = df[msk]

dtrain = xgb.DMatrix(df[factors].values, label=df.price, feature_names=factors)
tscv = TimeSeriesSplit(n_splits=5)

# per link below i need to use Upper Confidence Bound and add some alpha (square of stdev), otherwise it starts to loop
# https://github.com/fmfn/BayesianOptimization/issues/10 
gp_params = { 'alpha' : 400000}
XGB_BOpt.maximize(init_points=5, n_iter=10, acq='ucb', kappa=50, **gp_params)

Initialization
----------------------------------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |     alpha |   colsample_bytree |       eta |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 


/home/ilya/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    1 | 190m50s | -102906.15365 |    9.8231 |             0.5659 |    0.9139 |    4.7747 |           4.8645 |     10.2601 |            14.1032 |      0.9590 | 
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    2 | 249m34s | -189572.75000 |    6.8514 |             0.2937 |    0.1268 |    7.1847 |           4.4766 |     11.9052 |            14.0477 |      0.8435 | 
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
Stopping. Best iteration:
[27]	train-mae:40336.9+511.778	test-mae:40891.7+448.819

    3 | 05m29s | -40891.65104 |    3.0038 |             0.5818 |    0.9359 |    5.2934 |           0.3594 |      6.1628 |            11.6286 |      0.6394 | 
Multiple eval me

/home/ilya/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:581: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of groups for any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)


Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
Stopping. Best iteration:
[23]	train-mae:43109.9+443.094	test-mae:43404.8+392.459

    6 | 05m27s | -43404.79557 |    0.1484 |             0.6054 |    0.7472 |    0.2076 |           0.0101 |      5.5702 |             1.1016 |      0.6057 | 
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    7 | 230m18s | -120814.48958 |    6.8398 |             0.9315 |    0.7214 |    3.0585 |           4.1179 |     12.9399 |            18.1821 |      0.8707 | Warning: Test point chose at random due to repeated sample.

Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 50 rounds.
    8 | 2432m20s | -120814.48958 |    6.8398 |             0.9315 |    0.7214 |    3.0585 |           4.1179 |     1

In [ ]:
# not used - reset the variable
#new_init = { 'target': [], 'alpha': [], 'colsample_bytree': [], 'gamma': [], 'max_delta_step': [], 'max_depth': [], 'min_child_weight': [], 'subsample': [] }
new_init = init_values

# store resulting values to help seed the next run. make sure not to overwrite but add incrementally
# copy paste the print out of init_values into the cell above
for i in range(len(XGB_BOpt.res['all']['values'])):
    new_init['target'].append(XGB_BOpt.res['all']['values'][i])
    for k,v in XGB_BOpt.res['all']['params'][i].items():
        new_init[k].append(v)
    
print (new_init)

In [ ]:
%matplotlib inline
print(XGB_BOpt.res['max'])
(pd.DataFrame(XGB_BOpt.res['all']['values'])*-1.0).plot()

In [ ]:
import json
import requests
from cloudinary.uploader import upload
from cloudinary.utils import cloudinary_url
from cloudinary.api import delete_resources_by_tag, resources_by_tag


from slacker import Slacker
import os
# slack secrets (in your ~/.bashrc)
slack_webhook_url = os.environ.get('SLACK_WEBHOOK')
slacker = Slacker(os.environ.get('SLACK_TOKEN'))


def plot_rounds(plot):
    # uploads the graph to the web and returns the URL
    
    fig = plot.get_figure()
    fig.savefig('temp_plot.png')
    
    response = upload("temp_plot.png")
    url, options = cloudinary_url(response['public_id'],
        format = response['format'],
        crop = "fill")
    return url

def slack(text, url = None):
    print("Slacking: " + text)
    
    if url == None:
        data=json.dumps({"text": text})
    else:
        data = json.dumps( { "text": text, "attachments": [ { "fallback": "Model MAE"
                                           , "title": "Model Mean Average Error by Iteration ($)"
                                           , "image_url": url } ] } )
    
    response = requests.post(webhook_url, data , headers={'Content-Type': 'application/json'})
    if response.status_code != 200:
        raise ValueError('Request to slack returned an error %s, the response is:\n%s' % (response.status_code, response.text))

In [ ]:
result = pd.DataFrame(XGB_BOpt.res['all']['params'])
error = pd.Series(XGB_BOpt.res['all']['values']) * -1
error.name = 'test-mae-mean'
result = pd.concat([error, result], axis=1)
result.head(25)

url = plot_rounds(error.plot())
slack("Bayesian Search: Max params %s" % XGB_BOpt.res['max'], url)

file = 'PH-bayesian-parameters.csv'
result.to_csv(file)
slacker.files.upload(file, channels='#progress')